<a href="https://colab.research.google.com/github/jjAguil/Tareas-Simulacion/blob/main/Proyecto_Electronicos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelo de clasificacióin de clientes de electronicos

### Importación de librerias

In [ ]:
!pip install nltk
dbutils.library.restartPython()

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime, timedelta
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
pd.set_option('display.max_rows', 1000000)
pd.set_option('display.max_columns', 1000000)
##import nltk
import re
from sklearn.feature_extraction.text import TfidfVectorizer

### Identificación de electronicos y marcas

In [ ]:
celulares=spark.sql("""
select distinct id_cliente,billete,desc_prenda,tasa ,
case
    when desc_prenda like '%SAMSUNG%' OR desc_prenda like '%SAMSUN%' OR desc_prenda like '%SAMSUMG%' OR desc_prenda like '%SUMSUNG%' OR desc_prenda like '%GALAXI%' OR desc_prenda like '%GALAXY%' then 'SAMSUNG'
    when desc_prenda like '%APPLE%' OR desc_prenda like '%IPHONE%' OR desc_prenda like '%IPONE%' OR desc_prenda like '%IPOHE%' then 'IPHONE'
    when desc_prenda like '%XIAOMI%' then 'XIAOMI'
    when desc_prenda like '%LG%' then 'LG'
    when desc_prenda like '%MOTOROLA%' OR desc_prenda like '%MOTO%' OR desc_prenda like '%MOTOG%' then 'MOTOROLA'
    when desc_prenda like '%ONE%' AND desc_prenda like '%PLUS%' then 'ONE PLUS'
    when desc_prenda like '%SONY%' then 'SONY'
    when desc_prenda like '%HTC%' then 'HTC'
    when desc_prenda like '%HUAWEI%' OR desc_prenda like '%HUAWEY%' then 'HUAWEI'
    when desc_prenda like '%HISENSE%' then 'HISENSE'
    when desc_prenda like '%OPPO%' OR desc_prenda like '%RENO%' then 'OPPO'
    when desc_prenda like '%REDMI%' then 'REDMI'
    when desc_prenda like '%VIVO%' then 'VIVO'
    when desc_prenda like '%ASUS%' then 'ASUS'
    when desc_prenda like '%HONOR%' then 'HONOR'
    when desc_prenda like '%ZTE%' then 'ZTE'
    when desc_prenda like '%ALCATEL%' then 'ALCATEL'
    when desc_prenda like '%NOKIA%' then 'NOKIA'
    when desc_prenda like '%LANIX%' then 'LANIX'
    ELSE 'OTROS' end as MARCA
from deposito
where desc_prenda like '%CELULAR%'
AND (desc_prenda like 'CELULAR%'
OR desc_prenda like 'CEL%'
OR desc_prenda like 'TELEFONO%'
OR desc_prenda like 'TELÉFONO%'
OR desc_prenda like 'TEL%'
)
and tasa is not null
""")
celulares.createOrReplaceTempView("celulares")

In [ ]:
pantallas=spark.sql("""
select distinct id_cliente,billete,desc_prenda,tasa ,
  case
    when desc_prenda like '%SAMSUNG%' OR desc_prenda like '%SAMSUN%' OR desc_prenda like '%SAMSUMG%' OR desc_prenda like '%SUMSUNG%' then 'SAMSUNG'
    when desc_prenda like '%LG%' then 'LG'
    when desc_prenda like '%HISENSE%' then 'HISENSE'
    when desc_prenda like '%SONY%' then 'SONY'
    when desc_prenda like '%ROKU%' then 'ROKU'
    when desc_prenda like '%JVC%' then 'JVC'
    when desc_prenda like '%ATVIO%' then 'ATVIO'
    when desc_prenda like '%RCA%' then 'RCA'
    when desc_prenda like '%HKPRO%' then 'HKPRO'
    when desc_prenda like '%PHILIPS%' OR desc_prenda like '%PHILLIPS%' then 'PHILIPS'
    when desc_prenda like '%VIOS%' then 'VIOS'
    when desc_prenda like '%SANSUI%' then 'SANSUI'
    when desc_prenda like '%SHARP%' then 'SHARP'
    when desc_prenda like '%PANASONIC%' then 'PANASONIC'
    when desc_prenda like '%POLAROID%' then 'POLAROID'
    when desc_prenda like '%PIONEER%' OR desc_prenda like '%PIONER%' then 'PIONEER'
    when desc_prenda like '%DAEWOO%' then 'DAEWOO'
    when desc_prenda like '%SPELER%' then 'SPELER'
    when desc_prenda like '%HYUNDAI%' then 'HYUNDAI'
    when desc_prenda like '%HITACHI%' then 'HITACHI'
    when desc_prenda like '%MAKENA%' then 'MAKENA'
    when desc_prenda like '%WESTINGHOUSE%' then 'WESTINGHOUSE'
    ELSE 'OTROS' end as MARCA
from deposito
where desc_prenda like 'PANTALLA%'
and tasa is not null
""")
pantallas.createOrReplaceTempView("pantallas")

In [ ]:
laptop=spark.sql("""
select distinct id_cliente,billete,desc_prenda,tasa  ,
  case
    when desc_prenda like '%HP%' OR desc_prenda like '%PAVILION%' then 'HP'
    when desc_prenda like '%DELL%' or desc_prenda like '%INSPIRON%' OR desc_prenda like '%LATITUDE%' OR desc_prenda like '%INSPIRION%' then 'DELL'
    when desc_prenda like '%ACER%' OR desc_prenda like '%ASER%' OR desc_prenda like '%ACCER%' then 'ACER'
    when desc_prenda like '%ASUS%' then 'ASUS'
    when desc_prenda like '%MACBOOK%' OR desc_prenda like '%APPLE%' OR desc_prenda like '%MAC%' then 'MACBOOK'
    when desc_prenda like '%LENOVO%' OR desc_prenda like '%THINKPAD%' OR desc_prenda like '%LENNOVO%' OR desc_prenda like '%LENVO%' then 'LENOVO'
    when desc_prenda like '%HUAWEI%' OR desc_prenda like '%HUAWEY%' then 'HUAWEI'
    when desc_prenda like '%TOSHIBA%' OR desc_prenda like '%TOSHIBBA%' OR desc_prenda like '%TOCHIBA%' then 'TOSHIBA'
    when desc_prenda like '%SONY%' then 'SONY'
    when desc_prenda like '%SAMSUNG%' OR desc_prenda like '%SAMSUN%' OR desc_prenda like '%SAMSUMG%' OR desc_prenda like '%SUMSUNG%' then 'SAMSUNG'
    when desc_prenda like '%MSI%' then 'MSI'
    when desc_prenda like '%ALIENWARE%' OR desc_prenda like '%ALIEMWARE%' then 'ALIENWARE'
    when desc_prenda like '%LANIX%' then 'LANIX'
    when desc_prenda like '%GATEWAY%' then 'GATEWAY'
    ELSE 'OTROS' END AS MARCA
from deposito
where ((desc_prenda like '%LAP%'
AND desc_prenda like '%TOP%')
OR desc_prenda like '%MACBOOK%'
OR desc_prenda like '%NOTEBOOK%')
and tasa is not null
""")
laptop.createOrReplaceTempView("laptop")

In [ ]:
smartwatch=spark.sql("""
select distinct id_cliente,billete,desc_prenda,tasa
from deposito
where desc_prenda like '%WATCH%'
and desc_prenda like '%SMART%'
and tasa is not null
""")
smartwatch.createOrReplaceTempView("smartwatch")

In [ ]:
CONSOLAS=spark.sql("""
select distinct id_cliente,billete,desc_prenda,tasa
from deposito
where ((desc_prenda like '%PLAY%' AND desc_prenda like '%STATION%')
OR (desc_prenda like '%XBOX%')
OR desc_prenda like '%NINTENDO%'
OR (desc_prenda like '%STEAM%' AND desc_prenda like '%DECK%'))
AND desc_prenda NOT LIKE '%JUEGOS%'
AND desc_prenda NOT LIKE '%JGOS%'
AND desc_prenda NOT LIKE '%PARA%'
AND desc_prenda NOT LIKE 'CONTROL%'
AND desc_prenda NOT LIKE '%DISCOS%'
AND desc_prenda NOT LIKE '%ESTUCH%'
AND desc_prenda NOT LIKE '%VASOS%'
AND desc_prenda NOT LIKE 'CONTRL%'
and tasa is not null
""")
CONSOLAS.createOrReplaceTempView("CONSOLAS")

### Conjuntamos nuestras tablas

Unimos nuestra base madre con las banderas de los tipos de electronicos y marcas

In [ ]:
PP=spark.sql("""
select a.*
    ,case when b.billete is not null then b.desc_prenda
        when c.billete is not null then c.desc_prenda
        when d.billete is not null then d.desc_prenda
        when e.billete is not null then e.desc_prenda
        when f.billete is not null then f.desc_prenda
        else null end as desc_prenda
    ,case when b.billete is not null then b.tasa
        when c.billete is not null then c.tasa
        when d.billete is not null then d.tasa
        when e.billete is not null then e.tasa
        when f.billete is not null then f.tasa
        else null end as tasa
    ,case when b.billete is not null then 'CELULAR'
        when c.billete is not null then 'PANTALLA'
        when d.billete is not null then 'LAPTOP'
        when e.billete is not null then 'SMARTWATCH'
        when f.billete is not null then 'CONSOLA'
        else null end as ramo
    ,case when b.billete is not null then b.marca
        when c.billete is not null then c.marca
        when d.billete is not null then d.marca
        else null end as marca
from movimientos a
left join celulares b
  on a.cli_billete=b.billete
left join pantallas c
  on a.cli_billete=c.billete
left join laptop d
  on a.cli_billete=d.billete
left join smartwatch e
  on a.cli_billete=e.billete
left join CONSOLAS f
  on a.cli_billete=f.billete
where cli_movimiento='PP'
and (cli_billete in(select billete from celulares)
or cli_billete in(select billete from pantallas)
or cli_billete in(select billete from laptop)
or cli_billete in(select billete from smartwatch)
or cli_billete in(select billete from CONSOLAS)
)
and a.num_particion between 202001 and 202504
""")
PP.createOrReplaceTempView("PP")

### Identificamos prendas recuperadas, no recuperadas y actualmente en refrendo

In [ ]:
REC=spark.sql("""
select cli_id_cli,cli_billete,min(cli_dias_deposito) as dias_dp
from movimientos
where cli_billete in (select distinct cli_billete from PP)
and cli_movimiento in ('DS','VB','DEX','DSO','DSB','DB','DCS')
group by cli_id_cli,cli_billete
""")
REC.createOrReplaceTempView("REC")

In [ ]:
NO_REC=spark.sql("""
select cli_id_cli,cli_billete,min(cli_dias_deposito) as dias_dp
from movimientos
where cli_billete in (select distinct cli_billete from PP)
and cli_billete not in (select distinct cli_billete from REC)
and cli_movimiento in ('IA','CA','CV','VP','VPE')
group by cli_id_cli,cli_billete
""")
NO_REC.createOrReplaceTempView("NO_REC")

In [ ]:
REF=spark.sql("""
select cli_id_cli,cli_billete,min(cli_dias_deposito) as dias_dp
from movimientos
where cli_billete in (select distinct cli_billete from PP)
and cli_billete not in (select distinct cli_billete from REC)
and cli_billete not in (select distinct cli_billete from NO_REC)
and cli_movimiento in ('RF','APL','RFE')
group by cli_id_cli,cli_billete
""")
REF.createOrReplaceTempView("REF")

### Union de banderas

Unimos nuestras banderas y generamos nuestro resumen del descriptivo

In [ ]:
union_=spark.sql("""
select a.*,
  case
    when b.cli_billete is not null then b.dias_dp
    when c.cli_billete is not null then c.dias_dp
    when d.cli_billete is not null then d.dias_dp
    else null
  end as dias_dp,
  case
    when b.cli_billete is not null then cli_prestamo
    else 0
  end as saldo_rec,
  case
    when c.cli_billete is not null then cli_prestamo
    else 0
  end as saldo_no_rec,
  case
    when d.cli_billete is not null then cli_prestamo
    else 0
  end as saldo_ref
from PP a
left join REC b
on a.cli_billete=b.cli_billete
left join NO_REC c
on a.cli_billete=c.cli_billete
left join REF d
on a.cli_billete=d.cli_billete
where b.cli_billete is not null or c.cli_billete is not null or d.cli_billete is not null
""")
union_.createOrReplaceTempView("union_")

In [ ]:
resumen=spark.sql("""
select num_particion,ramo,marca,count(*) as prendas,sum(cli_prestamo) as prestamo, sum(cli_avaluo) as avaluo, sum(saldo_rec) as saldo_rec, sum(saldo_no_rec) as saldo_no_rec, sum(saldo_ref) as saldo_ref, sum(tasa) as tasa
from union_
group by num_particion,ramo,marca
""").toPandas()

In [ ]:
resumen

### Creación de variables de clientes

In [ ]:
%sql
declare or replace fecha_eje = '2025-12-30';
declare or replace cosecha_act = 202512

In [ ]:
MOVIMIENTOS_TOT_24M=spark.sql("""
SELECT *, datediff(month, date(cl_fecha_movimiento), fecha_eje) as meses_diff,
		datediff(day, date(cl_fecha_movimiento), fecha_eje) as Dias
FROM movimientos
WHERE datediff(month, date(cl_fecha_movimiento), fecha_eje) <=60
and datediff(month, date(cl_fecha_movimiento), fecha_eje) >= 0
and date(cl_fecha_movimiento)<=fecha_eje
and cli_id_cli!=0
and (cli_billete in (select distinct billete from celulares) or cli_billete in (select distinct billete from laptop) or cli_billete in (select distinct billete from pantallas) or cli_billete in (select distinct billete from smartwatch) or cli_billete in (select distinct billete from CONSOLAS))
and cli_movimiento in ('DS', 'VB', 'VP', 'CA', 'CV', 'IA', 'DEX', 'PP','RF')
and (cli_billete in (select distinct cli_billete from REC) OR cli_billete in (select distinct cli_billete from NO_REC))
""")
MOVIMIENTOS_TOT_24M.createOrReplaceTempView("MOVIMIENTOS_TOT_24M")

In [ ]:
emp_prom=spark.sql("""
SELECT cli_id_cli,avg(cli_prestamo) as emp_prom,count(distinct cli_billete) as empenios
from movimientos
where cli_movimiento='PP'
and cli_billete in (select distinct cli_billete from MOVIMIENTOS_TOT_24M)
and (cli_billete in (select distinct cli_billete from REC) OR cli_billete in (select distinct cli_billete from NO_REC))
group by cli_id_cli

""")
emp_prom.createOrReplaceTempView("emp_prom")

In [ ]:
dp_prom_bill = spark.sql("""
SELECT DISTINCT a.cli_id_cli,
       a.cli_billete,
       CASE
         WHEN  a.cli_billete in (
             SELECT DISTINCT cli_billete
             FROM REC
           )
         THEN d.dias_dp
         WHEN a.cli_billete in (
             SELECT DISTINCT cli_billete
             FROM NO_REC
           )
           THEN datediff(
             day,
             date(c.cl_fecha_movimiento),
             date(b.cl_fecha_movimiento)
           )
         ELSE 0

       END AS dias_dp
FROM MOVIMIENTOS_TOT_24M a
LEFT JOIN (
  SELECT cli_billete, min(cl_fecha_movimiento) AS cl_fecha_movimiento
  FROM movimientos
  WHERE cli_movimiento = 'PP'
  GROUP BY cli_billete
) b
  ON a.cli_billete = b.cli_billete
LEFT JOIN (
  SELECT cli_billete, min(cl_fecha_movimiento) AS cl_fecha_movimiento
  FROM movimientos
  WHERE cli_movimiento IN ('IA')
  GROUP BY cli_billete
) c
  ON a.cli_billete = c.cli_billete
LEFT JOIN REC d
  on a.cli_billete=d.cli_billete
LEFT JOIN NO_REC e
  on a.cli_billete=e.cli_billete
LEFT JOIN REF f
  on a.cli_billete=f.cli_billete
where a.cli_movimiento not in ('PP','RF')
and (a.cli_billete in (select distinct cli_billete from REC) OR a.cli_billete in (select distinct cli_billete from NO_REC))
""")
dp_prom_bill.createOrReplaceTempView("dp_prom_bill")

In [ ]:

dp_prom=spark.sql("""
SELECT  cli_id_cli,avg(dias_dp) as dp_prom
from dp_prom_bill
group by cli_id_cli
""")
dp_prom.createOrReplaceTempView("dp_prom")

In [ ]:
rec=spark.sql("""
SELECT a.cli_id_cli,a.cli_billete,a.cli_prestamo,
      case
        when b.cli_billete is not null then cli_prestamo
        else 0 end as recuperado
from MOVIMIENTOS_TOT_24M a
  left join (select distinct cli_billete
              from movimientos
                where cli_movimiento in ('DS','DEX','VB')) b
  on a.cli_billete=b.cli_billete
  where (a.cli_billete in (select distinct cli_billete from REC) OR a.cli_billete in (select distinct cli_billete from NO_REC))
""")
rec.createOrReplaceTempView("rec")

In [ ]:
rec_prom=spark.sql("""
SELECT cli_id_cli,sum(recuperado)/sum(cli_prestamo) as rec_prom
from rec
group by cli_id_cli
""")
rec_prom.createOrReplaceTempView("rec_prom")

In [ ]:
deposito=spark.sql("""
SELECT cli_id_cli,sum(cli_prestamo) as deposito
from movimientos
where cli_movimiento='DP'
and date(cl_fecha_movimiento) = fecha_eje
and cli_billete in (select distinct cli_billete from MOVIMIENTOS_TOT_24M)
group by cli_id_cli
""")
deposito.createOrReplaceTempView("deposito")

In [ ]:
variables=spark.sql("""
SELECT distinct a.cli_id_cli,b.empenios,b.emp_prom,c.dp_prom,d.rec_prom,e.deposito
from MOVIMIENTOS_TOT_24M a
left join emp_prom b
on a.cli_id_cli=b.cli_id_cli
left join dp_prom c
on a.cli_id_cli=c.cli_id_cli
left join rec_prom d
on a.cli_id_cli=d.cli_id_cli
left join deposito e
on a.cli_id_cli=e.cli_id_cli
""")
variables.createOrReplaceTempView("variables")

In [ ]:
variables.write.mode('overwrite').option('mergeSchema', 'true').saveAsTable('variables_modelo_electronicos')

### Creación del modelo cluster

Importamos librerias

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans, DBSCAN
from sklearn.decomposition import PCA

Cargamos la base que usaremos y la limpiamos

In [ ]:
base=spark.table('variables_modelo_electronicos')
base.createOrReplaceTempView("base")

In [ ]:
base_limpia=spark.sql("""
select cli_id_cli,emp_prom,abs(dp_prom) as dp_prom,rec_prom,empenios
from base
where dp_prom is not null
""").toPandas()

In [ ]:
base_limpia2=base_limpia[['emp_prom','dp_prom','rec_prom','empenios']].copy()

Identificación de nulos

In [ ]:
mask = base_limpia2['emp_prom'].isnull()
mask2 = base_limpia2['dp_prom'].isnull()
mask3 = base_limpia2['rec_prom'].isnull()
mask4 = base_limpia2['empenios'].isnull()
filtered_df = base_limpia2[mask | mask2 | mask3 | mask4]

In [ ]:
filtered_df

Declaramos los modelos a usar

In [ ]:
kmeans = KMeans(n_clusters=8, random_state=42)
dbscan = DBSCAN(eps=0.05, min_samples=8)

Entrenamos los modelos

In [ ]:
kmeans.fit(base_limpia2)

In [ ]:
dbscan.fit(base_limpia2)

Declaramos resultados

In [ ]:
kmeans.cluster_centers_

In [ ]:
kmeans.labels_

Realizamos reducción dimensional para poder visualizar los resultados

In [ ]:
pca= PCA(n_components=2)

In [ ]:
pca.fit(base_limpia2)

In [ ]:
base_limpia2_reducida=pca.transform(base_limpia2)

visualizamos resultados

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, sharey=True,  figsize=(10, 6))
ax1.set_title('K-means')
ax1.scatter(base_limpia2_reducida[:,0], base_limpia2_reducida[:,1], c=kmeans.labels_,cmap='rainbow')
ax2.set_title('DBSCAN')
ax2.scatter(base_limpia2_reducida[:,0], base_limpia2_reducida[:,1], c=dbscan.labels_,cmap='rainbow')
plt.show()

### Modelo elegido
### K-means

Ahora sacaremos los estadisticos por cada cluster del modelo en cada variable

In [ ]:
base_res=base_limpia2.copy()
base_res['cluster']=kmeans.labels_

In [ ]:
#Clientes A
filtro1=base_res['cluster']==0
est_c0=base_res[filtro1]
var=est_c0.describe()
var['rend_prom']=(var['rec_prom']*var['dp_prom']*(3.79/30))/100
var['rend_prom'] = var['rend_prom'].apply(lambda x: f"{x:.2%}")
var

In [ ]:
#Clientes F
filtro2=base_res['cluster']==1
est_c1=base_res[filtro2]
var=est_c1.describe()
var['rend_prom']=(var['rec_prom']*var['dp_prom']*(3.79/30))/100
var['rend_prom'] = var['rend_prom'].apply(lambda x: f"{x:.2%}")
var

In [ ]:
# Clientes B
filtro3=base_res['cluster']==2
est_c2=base_res[filtro3]
var=est_c2.describe()
var['rend_prom']=(var['rec_prom']*var['dp_prom']*(3.79/30))/100
var['rend_prom'] = var['rend_prom'].apply(lambda x: f"{x:.2%}")
var

In [ ]:
#Clientes D
filtro4=base_res['cluster']==3
est_c3=base_res[filtro4]
var=est_c3.describe()
var['rend_prom']=(var['rec_prom']*var['dp_prom']*(3.79/30))/100
var['rend_prom'] = var['rend_prom'].apply(lambda x: f"{x:.2%}")
var

In [ ]:
# Clientes S
filtro5=base_res['cluster']==4
est_c4=base_res[filtro5]
var=est_c4.describe()
var['rend_prom']=(var['rec_prom']*var['dp_prom']*(3.79/30))/100
var['rend_prom'] = var['rend_prom'].apply(lambda x: f"{x:.2%}")
var

In [ ]:
# Clientes E
filtro6=base_res['cluster']==5
est_c5=base_res[filtro6]
var=est_c5.describe()
var['rend_prom']=(var['rec_prom']*var['dp_prom']*(3.79/30))/100
var['rend_prom'] = var['rend_prom'].apply(lambda x: f"{x:.2%}")
var

In [ ]:
# Clientes C
filtro7=base_res['cluster']==6
est_c6=base_res[filtro7]
var=est_c6.describe()
var['rend_prom']=(var['rec_prom']*var['dp_prom']*(3.79/30))/100
var['rend_prom'] = var['rend_prom'].apply(lambda x: f"{x:.2%}")
var

In [ ]:
# Clientes SS
filtro8=base_res['cluster']==7
est_c7=base_res[filtro8]
var=est_c7.describe()
var['rend_prom']=(var['rec_prom']*var['dp_prom']*(3.79/30))/100
var['rend_prom'] = var['rend_prom'].apply(lambda x: f"{x:.2%}")
var

# Evaluamos nuestro modelo con los cluster generados y la nueva tabla de aforos

In [ ]:
base_nva=spark.table('clientes_electr_202501')
base_nva.createOrReplaceTempView("base_nva")

In [ ]:
(base_nva.count(), len(base_nva.columns))

In [ ]:
base_limpia_nva=spark.sql("""
select cli_id_cli,emp_prom,abs(dp_prom) as dp_prom,rec_prom,empenios
from base_nva
""").toPandas()

In [ ]:
media_dias=base_limpia_nva['dp_prom'].mean()
base_limpia_nva['dp_prom']=base_limpia_nva['dp_prom'].fillna(media_dias)

In [ ]:
base_limpia2_nva=base_limpia_nva[['emp_prom','dp_prom','rec_prom','empenios']].copy()

In [ ]:
base_limpia2_nva.shape

In [ ]:
m2=kmeans.predict(base_limpia2_nva)

In [ ]:
base_limpia2_nva['cluster_']=m2

In [ ]:
df=base_nva.toPandas()
df.head()

In [ ]:
db=pd.concat([base_limpia_nva, base_limpia2_nva[['cluster_']]], axis=1)
db.tail()

In [ ]:
spark_df = spark.createDataFrame(db)
spark_df.createOrReplaceTempView('db')

In [ ]:
clasificaciones=spark.sql("""
select *,
    case when cluster_=0 then 'A'
    when cluster_=1 then 'F'
    when cluster_=2 then 'B'
    when cluster_=3 then 'D'
    when cluster_=4 then 'S'
    when cluster_=5 then 'C'
    when cluster_=6 then 'E'
    when cluster_=7 then 'SS'
    else 'N' end as clasificacion
from db
""")
clasificaciones.createOrReplaceTempView('clasificaciones')

In [ ]:
evaluacion=spark.sql("""
select a.* ,
    case
      when b.clasificacion is not null then clasificacion
      else 'N' end as clasificacion,
    case
      when c.cli_billete is null then 0
      else a.cli_prestamo end as recuperado,
    case
      when a.ramo='CELULAR' and a.marca in ('HTC','VIVO','HONOR','HUAWEI','OPPO') then 'Grupo1'
      when a.ramo='CELULAR' and a.marca in ('SAMSUNG','OTROS','ZTE','SONY','ALCATEL') then 'Grupo2'
      when a.ramo='CELULAR' and a.marca in ('MOTOROLA','XIAOMI','REDMI','LANIX','IPHONE') then 'Grupo3'
      when a.ramo='CELULAR' and a.marca in ('NOKIA','LG','HISENSE','ONE PLUS','ASUS') then 'Grupo4'
      when a.ramo='PANTALLA' and a.marca in ('WESTINGHOUSE','ROKU','MAKENA','POLAROID','HYUNDAI','JVC') then 'Grupo1'
      when a.ramo='PANTALLA' and a.marca in ('DAEWOO','HISENSE','HITACHI','OTROS','RCA','SPELER') then 'Grupo2'
      when a.ramo='PANTALLA' and a.marca in ('PHILIPS','ATVIO','LG','SHARP','SAMSUNG','SANSUI') then 'Grupo3'
      when a.ramo='PANTALLA' and a.marca in ('HKPRO','PIONEER','VIOS','PANASONIC','SONY') then 'Grupo4'
      when a.ramo='LAPTOP' and a.marca in ('GATEWAY','DELL','SAMSUNG','LANIX') then 'Grupo1'
      when a.ramo='LAPTOP' and a.marca in ('OTROS','ALIENWARE','LENOVO','HUAWEI') then 'Grupo2'
      when a.ramo='LAPTOP' and a.marca in ('ASUS','ACER','MACBOOK','MSI') then 'Grupo3'
      when a.ramo='LAPTOP' and a.marca in ('HP','TOSHIBA','SONY') Then 'Grupo4'
      else null end as grupos
from PP a
left join clasificaciones b
on a.cli_id_cli=b.cli_id_cli
left join (
  select distinct cli_billete from movimientos
    where cli_movimiento in ('DS','VB','DEX','DSO','DSB','DB','DCS')
) c
on a.cli_billete=c.cli_billete
where a.ramo in ('CELULAR','PANTALLA','LAPTOP')
and a.num_particion=202501
""")
evaluacion.createOrReplaceTempView('evaluacion')

ahora cargamos los aforos propuestos

In [ ]:
data = [
    {'grupo': 'Grupo1', 'SS': 0.59, 'S': 0.57, 'A': 0.55, 'B': 0.53, 'C': 0.51, 'D': 0.49, 'E': 0.47, 'F': 0.45, 'N': 0.51},
    {'grupo': 'Grupo2', 'SS': 0.61, 'S': 0.59, 'A': 0.57, 'B': 0.55, 'C': 0.53, 'D': 0.51, 'E': 0.49, 'F': 0.47, 'N': 0.53},
    {'grupo': 'Grupo3', 'SS': 0.63, 'S': 0.61, 'A': 0.59, 'B': 0.57, 'C': 0.55, 'D': 0.53, 'E': 0.51, 'F': 0.49, 'N': 0.55},
    {'grupo': 'Grupo4', 'SS': 0.65, 'S': 0.63, 'A': 0.61, 'B': 0.59, 'C': 0.57, 'D': 0.55, 'E': 0.53, 'F': 0.51, 'N': 0.57}
]

df = pd.DataFrame(data)
celu_af = spark.createDataFrame(df)
celu_af_long = celu_af.unpivot(
    ids="grupo",
    values=["SS", "S", "A", "B", "C", "D", "E", "F", "N"],
    variableColumnName="clasificacion",
    valueColumnName="aforo"
)
celu_af_long.createOrReplaceTempView('celu_af_long')
display(celu_af_long)

In [ ]:
data_af = [
    {'grupo': 'Grupo1', 'SS': 0.56, 'S': 0.54, 'A': 0.52, 'B': 0.50, 'C': 0.48, 'D': 0.46, 'E': 0.44, 'F': 0.42, 'N': 0.48},
    {'grupo': 'Grupo2', 'SS': 0.58, 'S': 0.56, 'A': 0.54, 'B': 0.52, 'C': 0.50, 'D': 0.48, 'E': 0.46, 'F': 0.44, 'N': 0.50},
    {'grupo': 'Grupo3', 'SS': 0.60, 'S': 0.58, 'A': 0.56, 'B': 0.54, 'C': 0.52, 'D': 0.50, 'E': 0.48, 'F': 0.46, 'N': 0.52},
    {'grupo': 'Grupo4', 'SS': 0.62, 'S': 0.60, 'A': 0.58, 'B': 0.56, 'C': 0.54, 'D': 0.52, 'E': 0.50, 'F': 0.48, 'N': 0.54}
]

pantalla_af = spark.createDataFrame(pd.DataFrame(data_af))
pantalla_af_long = pantalla_af.unpivot(
    ids="grupo",
    values=["SS", "S", "A", "B", "C", "D", "E", "F", "N"],
    variableColumnName="clasificacion",
    valueColumnName="aforo"
)
pantalla_af_long.createOrReplaceTempView('pantalla_af_long')
display(pantalla_af_long)

In [ ]:
data_laptop_af = [
    {'grupo': 'Grupo1', 'SS': 0.57, 'S': 0.56, 'A': 0.53, 'B': 0.51, 'C': 0.49, 'D': 0.47, 'E': 0.45, 'F': 0.43, 'N': 0.49},
    {'grupo': 'Grupo2', 'SS': 0.59, 'S': 0.58, 'A': 0.55, 'B': 0.53, 'C': 0.51, 'D': 0.49, 'E': 0.47, 'F': 0.45, 'N': 0.51},
    {'grupo': 'Grupo3', 'SS': 0.61, 'S': 0.60, 'A': 0.57, 'B': 0.55, 'C': 0.53, 'D': 0.51, 'E': 0.49, 'F': 0.47, 'N': 0.53},
    {'grupo': 'Grupo4', 'SS': 0.63, 'S': 0.62, 'A': 0.59, 'B': 0.57, 'C': 0.55, 'D': 0.53, 'E': 0.51, 'F': 0.49, 'N': 0.55}
]

laptop_af = spark.createDataFrame(pd.DataFrame(data_laptop_af))
laptop_af_long = laptop_af.unpivot(
    ids="grupo",
    values=["SS", "S", "A", "B", "C", "D", "E", "F", "N"],
    variableColumnName="clasificacion",
    valueColumnName="aforo"
)
laptop_af_long.createOrReplaceTempView('laptop_af_long')
display(laptop_af_long)

In [ ]:
evaluacion=spark.sql("""
select a.* ,
    case when a.ramo='CELULAR' then b.aforo
    when a.ramo='PANTALLA' then c.aforo
    when a.ramo='LAPTOP' then d.aforo
    else null end as aforo_sim
from evaluacion a
left join celu_af_long b
on a.grupos=b.grupo and a.clasificacion=b.clasificacion
left join pantalla_af_long c
on a.grupos=c.grupo and a.clasificacion=c.clasificacion
left join laptop_af_long d
on a.grupos=d.grupo and a.clasificacion=d.clasificacion
""")
evaluacion.createOrReplaceTempView('evaluacion')

In [ ]:
evaluacion=spark.sql("""
select *,
    (cli_avaluo)*aforo_sim as prestamo_sim,
    case when recuperado>0 then prestamo_sim else 0 end as recuperado_sim
from evaluacion
""")
evaluacion.createOrReplaceTempView('evaluacion')

In [ ]:
resumen_final=spark.sql("""
select clasificacion,ramo,grupos,
  sum(cli_prestamo) as prestamo_orig,
  sum(prestamo_sim) as prestamo_sim,
  sum(cli_avaluo) as avaluo,
  sum(tasa) as tasa_sum,
  sum(recuperado) as recuperacion,
  sum(recuperado_sim) as recuperacion_sim,
  count(distinct cli_id_cli) as clientes
from evaluacion
group by clasificacion,ramo,grupos
""").toPandas()

In [ ]:
resumen_final

### Solo identificamos los pesos de los empeños de cada tipo de dispositivo

In [ ]:
pareto=spark.sql("""
select num_particion,ramo,count(*) as prendas,sum(cli_prestamo) as prestamo, sum(cli_avaluo) as avaluo
from PP
group by num_particion,ramo
order by num_particion,ramo
""").toPandas()
pareto

### Proceso de Tokenización

In [ ]:
prueba=spark.sql("""
select a.num_particion,a.cl_fecha_movimiento, a.cli_id_cli, a.cli_billete,c.id,c.descripcion
from movimientos a
left join to_partida_mercancia b on a.cli_billete=b.folio
left join nto_avaluo c on b.avaluo=c.id
left join tab_transacciones_ofertas d
on d.idPartida=a.cli_billete
where a.cli_movimiento='PP'
and ((c.descripcion like '%LAP%'
AND c.descripcion like '%TOP%')
OR c.descripcion like '%MACBOOK%'
OR c.descripcion like '%NOTEBOOK%')
""").toPandas()

In [ ]:
pattern = r'''(?x)                    #Expresiones regulares detalladas
              (?:[A-Z]\.)+            #Abreviaciones
              | \w+(?:-\w+)*          #Guiones
              | \$?\d+(?:\.\d+)?%?    #Monedas y porcentajes
              | \.\.\.                #Elipsis
              | [][.,;"'?():-_`]      #Tokens separados por ],[
'''

In [ ]:
dfn = prueba.descripcion.dropna()
#dfn = dfn.apply(lambda x: nltk.regexp_tokenize(x, pattern))

In [ ]:
# Join the filtered list into a single string
text = " ".join(dfn)

# Tokenize the text using the regular expression pattern
tokens = nltk.regexp_tokenize(text, pattern)

In [ ]:
word_freq=nltk.FreqDist(tokens)
word_freq.most_common()